Сингулярное разложение — определенного типа разложение прямоугольной матрицы, имеющее широкое применение при решении многих прикладных задач.

Неотрицательное вещественное число σ называется сингулярным числом матрицы M, когда существуют два вектора единичной длины u и v такие, что: Mv = σu и M'u = σv, где M' - сопряженно-транспонированная матрица (матрица полученная из исходной матрицы транспонированием и заменой каждого элемента комплексно-сопряженным ему). u
u и v называются, соответственно, левым сингулярным вектором и правым сингулярным вектором, соответствующим сингулярному числу σ.

Сингулярным разложением матрицы M порядка m×n является разложение следующего вида:
М = UEV', где E — матрица размера m×n с неотрицательными элементами, у которой элементы, лежащие на главной диагонали — это сингулярные числа (а все элементы, не лежащие на главной диагонали, являются нулевыми), а матрицы U (порядка m) и V (порядка n) — это две унитарные матрицы, состоящие из левых и правых сингулярных векторов соответственно (а V' — это сопряжённо-транспонированная матрица к V).

In [29]:
import numpy as np
import pandas as pd

In [30]:
def data_transform() -> np.ndarray:
    '''
    Считывает данные из таблицы и преобразует в массив

    Returns
    -------
    matrix[:5, :]
    '''
    df = pd.read_csv(
        '/Users/Slava/Документы/IT/Курс_по_DS/Jupiter(ДЗ)/Задача рекоммендации(SVD разложение)/ratings.csv'
    )
    print(df)
    df.drop('timestamp', axis=1, inplace=True)
    
    matrix = df.pivot_table(index='userId',
                            columns='movieId',
                            values='rating',
                            fill_value=0).values
    return matrix[:5, :]

In [31]:
data_transform()

        userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]


array([[4., 0., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.]])

In [32]:
def data_film_transform() -> dict:
    '''
    Считывает данные из таблицы и преобразует их в отсортированный по значению словарь

    Returns
    -------
    matrix_film_sorted
    '''
    df_film = pd.read_csv(
        '/Users/Slava/Документы/IT/Курс_по_DS/Jupiter(ДЗ)/Задача рекоммендации(SVD разложение)/movies.csv'
    )
    print(df_film)
    df_film.drop('genres', axis=1, inplace=True)

    matrix_film = df_film.pivot_table(index='title',
                                      values='movieId').to_dict()
    # сортировка словаря по значению
    matrix_film_sorted = {}
    sorted_keys = sorted(matrix_film['movieId'],
                         key=matrix_film['movieId'].get)
    for w in sorted_keys:
        matrix_film_sorted[w] = int(matrix_film['movieId'][w])

    return matrix_film_sorted

In [33]:
data_film_transform()

      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                                  Comedy|Romance  
3                            Comedy|Drama|Romance  
4                  

{'Toy Story (1995)': 1,
 'Jumanji (1995)': 2,
 'Grumpier Old Men (1995)': 3,
 'Waiting to Exhale (1995)': 4,
 'Father of the Bride Part II (1995)': 5,
 'Heat (1995)': 6,
 'Sabrina (1995)': 7,
 'Tom and Huck (1995)': 8,
 'Sudden Death (1995)': 9,
 'GoldenEye (1995)': 10,
 'American President, The (1995)': 11,
 'Dracula: Dead and Loving It (1995)': 12,
 'Balto (1995)': 13,
 'Nixon (1995)': 14,
 'Cutthroat Island (1995)': 15,
 'Casino (1995)': 16,
 'Sense and Sensibility (1995)': 17,
 'Four Rooms (1995)': 18,
 'Ace Ventura: When Nature Calls (1995)': 19,
 'Money Train (1995)': 20,
 'Get Shorty (1995)': 21,
 'Copycat (1995)': 22,
 'Assassins (1995)': 23,
 'Powder (1995)': 24,
 'Leaving Las Vegas (1995)': 25,
 'Othello (1995)': 26,
 'Now and Then (1995)': 27,
 'Persuasion (1995)': 28,
 'City of Lost Children, The (Cité des enfants perdus, La) (1995)': 29,
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': 30,
 'Dangerous Minds (1995)': 31,
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)':

In [34]:
def svd_decomposition(matrix: np.ndarray, k: int=5):
    '''
    Осуществялет SVD разложение и на его основе восстанавливает новую матрицу

    Returns
    -------
    new_matrix[:, :5000]
    '''
    U, S, Vt = np.linalg.svd(matrix)
    new_matrix = U[:, 0:k] @ np.diag(S[:k]) @ Vt[0:k, :]
    return new_matrix[:, :5000]

In [35]:
a = data_transform()
svd_decomposition(matrix=a, k=5), svd_decomposition(matrix=a, k=5).shape

        userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]


(array([[ 4.00000000e+00,  2.44593103e-32,  4.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 8.58579700e-16, -5.12297365e-32,  1.07119175e-16, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 7.42461648e-16,  1.61831732e-17, -3.03923553e-15, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 2.05143519e-14,  7.06519879e-17,  3.84783351e-16, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 4.00000000e+00,  1.15154900e-15,  7.06845606e-16, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 (5, 5000))

In [36]:
def svd_result(new_matrix: np.ndarray, matrix_film_sorted: dict) -> list:
    '''
    На основании новой матрицы производит сопоставление с фильмами и
    сортировку по Топ-3 лучших фильмов для пользователя

    Returns
    -------
    lst
    '''
    lst = []
    for i in range(new_matrix.shape[0]):
        my_dict = {}
        for j, d in zip(new_matrix[i, :], matrix_film_sorted):
            my_dict[j] = d

        # сортировка словаря по ключу
        sorted(my_dict.items())
        my_dict_sort = dict(sorted(my_dict.items()))

        l = list(my_dict_sort.values())
        lst.append(l[::-1][:3])
    return lst

In [37]:
b = svd_decomposition(matrix=a, k=5)
d = data_film_transform()
c = svd_result(new_matrix=b, matrix_film_sorted=d)
n = 1

for i in c:
    print(f'Топ-3 фильма для пользователя {n} - {i}')
    n += 1

      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                                  Comedy|Romance  
3                            Comedy|Drama|Romance  
4                  